# Analyse de la structure de bandes électroniques du RbLaS2

## Téléchargement de la structure de bande 

In [1]:
from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifWriter
from matplotlib import pyplot as plt 
from tabulate import tabulate
from pymatgen.electronic_structure import bandstructure

In [2]:
#mission 8
m = MPRester("H45GsyS7rpfkZzFI")
bandstructure = m.get_bandstructure_by_material_id("mp-9361")

## Analyse de la bande interdite 

In [3]:
bandgap = bandstructure.get_band_gap()
print("Energie de la bande interdite : "+str(bandgap['energy'])+" eV")
print("Bande interdite directe : "+str(bandgap['direct']))
print("Transition dans la bande interdite : "+str(bandgap['transition']))
print("\n")

Energie de la bande interdite : 2.6856999999999998 eV
Bande interdite directe : False
Transition dans la bande interdite : \Gamma-F




Comme la bande interdite a une énergie supérieure à 2 eV, on peut dire du cristal qu'il a un comportement isolant au niveau électrique.

En outre, la bande interdite est indirecte. Cela veut dire que le minimum de la bande de conduction et le maximum de la bande de valence correspondent à des vecteurs d'onde différents. En effet le vecteur d'onde évolue selon une direction $\Gamma-F$ dans la bande interdite.